# Project of ana and leo

In [14]:
import pandas as pd
import statistics
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import function
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

warnings.filterwarnings('ignore')

## Data loading

In [2]:
df = pd.read_csv("data/Sampled_BPIC17_offer_log.csv")

In [3]:
df

,index,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,(case) Accepted,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventID,EventOrigin,OfferID,lifecycle:transition
0,0,Offer_247135719,O_Create Offer,User_17,2016/01/02 18:17:05.720,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,Created,Offer_247135719,Offer,NaN,complete
1,1,Offer_247135719,O_Created,User_17,2016/01/02 18:17:08.762,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,statechange,OfferState_124849367,Offer,Offer_247135719,complete
2,2,Offer_247135719,O_Sent (online only),User_17,2016/01/02 18:19:21.330,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,statechange,OfferState_440662877,Offer,Offer_247135719,complete
3,3,Offer_247135719,O_Cancelled,User_17,2016/01/02 18:21:26.034,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,statechange,OfferState_591416028,Offer,Offer_247135719,complete
4,4,Offer_941964966,O_Create Offer,User_17,2016/01/02 18:21:42.022,Variant 1,1,True,Application_196483749,0,4100.0,201.76,57,10000.0,False,Created,Offer_941964966,Offer,NaN,complete
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22404,22404,Offer_1064426652,O_Create Offer,User_23,2016/02/18 22:50:01.446,Variant 2,2,True,Application_1574359134,767,6000.0,118.56,59,6000.0,True,Created,Offer_1064426652,Offer,NaN,complete
22405,22405,Offer_1064426652,O_Created,User_23,2016/02/18 22:50:02.655,Variant 2,2,True,Application_1574359134,767,6000.0,118.56,59,6000.0,True,statechange,OfferState_189916093,Offer,Offer_1064426652,complete
22406,22406,Offer_1064426652,O_Sent (mail and online),User_23,2016/02/18 22:50:22.174,Variant 2,2,True,Application_1574359134,767,6000.0,118.56,59,6000.0,True,statechange,OfferState_24747308,Offer,Offer_1064426652,complete
22407,22407,Offer_1064426652,O_Returned,User_117,2016/02/29 19:08:56.200,Variant 2,2,True,Application_1574359134,767,6000.0,118.56,59,6000.0,True,statechange,OfferState_1809485991,Offer,Offer_1064426652,complete


In [4]:
df.info

<bound method DataFrame.info of        index           Case ID                  Activity  Resource  \
0          0   Offer_247135719            O_Create Offer   User_17   
1          1   Offer_247135719                 O_Created   User_17   
2          2   Offer_247135719      O_Sent (online only)   User_17   
3          3   Offer_247135719               O_Cancelled   User_17   
4          4   Offer_941964966            O_Create Offer   User_17   
...      ...               ...                       ...       ...   
22404  22404  Offer_1064426652            O_Create Offer   User_23   
22405  22405  Offer_1064426652                 O_Created   User_23   
22406  22406  Offer_1064426652  O_Sent (mail and online)   User_23   
22407  22407  Offer_1064426652                O_Returned  User_117   
22408  22408  Offer_1064426652                O_Accepted  User_102   

            Complete Timestamp    Variant  Variant index  (case) Accepted  \
0      2016/01/02 18:17:05.720  Variant 8         

## Data preprocessing

In [5]:
input = {
    "Case ID" : "caseid",
    "Activity" : "activity" ,
    "Reouce" : "resource" ,
    "Complete Timestamp": "ts",
    "(case) Accepted" : "y"  
}

# Change columns name
df.rename(columns=input,inplace = True)

# Timestamp value in df
df["ts"] = pd.to_datetime(df["ts"])

df["dt"] = function.dt(df)


In [6]:
df.isna().sum()*100/df.count()

index                            0.000000
caseid                           0.000000
activity                         0.000000
Resource                         0.000000
ts                               0.000000
Variant                          0.000000
Variant index                    0.000000
y                                0.000000
(case) ApplicationID             0.000000
(case) CreditScore               0.000000
(case) FirstWithdrawalAmount     0.000000
(case) MonthlyCost               0.000000
(case) NumberOfTerms             0.000000
(case) OfferedAmount             0.000000
(case) Selected                  0.000000
Action                           0.000000
EventID                          0.000000
EventOrigin                      0.000000
OfferID                         28.720777
lifecycle:transition             0.000000
dt                               0.000000
dtype: float64

In [9]:
df.isnull().sum()*100/df.count()



index                            0.000000
caseid                           0.000000
activity                         0.000000
Resource                         0.000000
ts                               0.000000
Variant                          0.000000
Variant index                    0.000000
y                                0.000000
(case) ApplicationID             0.000000
(case) CreditScore               0.000000
(case) FirstWithdrawalAmount     0.000000
(case) MonthlyCost               0.000000
(case) NumberOfTerms             0.000000
(case) OfferedAmount             0.000000
(case) Selected                  0.000000
Action                           0.000000
EventID                          0.000000
EventOrigin                      0.000000
OfferID                         28.720777
lifecycle:transition             0.000000
dt                               0.000000
dtype: float64

In [25]:
df['OfferID'] = df['OfferID'].fillna('Unknown')

In [28]:
df['(case) Selected'] = df['(case) Selected'].astype(int)
df['y'] = df['y'].astype(int)

In [29]:
df

,index,caseid,activity,Resource,ts,Variant,Variant index,y,(case) ApplicationID,(case) CreditScore,...,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventID,EventOrigin,OfferID,lifecycle:transition,dt
0,0,Offer_247135719,O_Create Offer,User_17,2016-01-02 18:17:05.720,Variant 8,8,1,Application_196483749,0,...,201.76,57,10000.0,0,Created,Offer_247135719,Offer,Unknown,complete,0 days 00:00:03.042000
1,1,Offer_247135719,O_Created,User_17,2016-01-02 18:17:08.762,Variant 8,8,1,Application_196483749,0,...,201.76,57,10000.0,0,statechange,OfferState_124849367,Offer,Offer_247135719,complete,0 days 00:02:12.568000
2,2,Offer_247135719,O_Sent (online only),User_17,2016-01-02 18:19:21.330,Variant 8,8,1,Application_196483749,0,...,201.76,57,10000.0,0,statechange,OfferState_440662877,Offer,Offer_247135719,complete,0 days 00:02:04.704000
3,3,Offer_247135719,O_Cancelled,User_17,2016-01-02 18:21:26.034,Variant 8,8,1,Application_196483749,0,...,201.76,57,10000.0,0,statechange,OfferState_591416028,Offer,Offer_247135719,complete,0 days 00:02:04.704000
4,4,Offer_941964966,O_Create Offer,User_17,2016-01-02 18:21:42.022,Variant 1,1,1,Application_196483749,0,...,201.76,57,10000.0,0,Created,Offer_941964966,Offer,Unknown,complete,0 days 00:00:01.551000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22404,22404,Offer_1064426652,O_Create Offer,User_23,2016-02-18 22:50:01.446,Variant 2,2,1,Application_1574359134,767,...,118.56,59,6000.0,1,Created,Offer_1064426652,Offer,Unknown,complete,0 days 00:00:01.209000
22405,22405,Offer_1064426652,O_Created,User_23,2016-02-18 22:50:02.655,Variant 2,2,1,Application_1574359134,767,...,118.56,59,6000.0,1,statechange,OfferState_189916093,Offer,Offer_1064426652,complete,0 days 00:00:19.519000
22406,22406,Offer_1064426652,O_Sent (mail and online),User_23,2016-02-18 22:50:22.174,Variant 2,2,1,Application_1574359134,767,...,118.56,59,6000.0,1,statechange,OfferState_24747308,Offer,Offer_1064426652,complete,10 days 20:18:34.026000
22407,22407,Offer_1064426652,O_Returned,User_117,2016-02-29 19:08:56.200,Variant 2,2,1,Application_1574359134,767,...,118.56,59,6000.0,1,statechange,OfferState_1809485991,Offer,Offer_1064426652,complete,0 days 04:34:49.631000


## Logs encoding

i think: activity - onehot?, variant - one hot, action, onehot, event origin - onehot, liofecycle - one hot

## Model selection

## Evaluation of results